In [1]:
import pathlib
import datetime
import os

In [19]:
import iris
import cf_units

In [3]:
import pandas

In [4]:
import xarray

In [5]:
import matplotlib
import matplotlib.pyplot

In [6]:
data_root = pathlib.Path('/scratch/shaddad/precip_rediagnosis/train_202207/')
event_name = '202110_nswws_amber_oct'
event_dir = data_root / event_name

In [7]:
mg_dir = event_dir / 'mogreps-g'
radar_dir = event_dir / 'radar' 

In [8]:
event_dir

PosixPath('/scratch/shaddad/precip_rediagnosis/train_202207/202110_nswws_amber_oct')

In [10]:
validity_times = pandas.date_range(
    start=datetime.datetime(2021,10,19,18,0), 
    end=datetime.datetime(2021,10,21,0,0), 
    freq=datetime.timedelta(hours=6),
).to_pydatetime()
validity_times

array([datetime.datetime(2021, 10, 19, 18, 0),
       datetime.datetime(2021, 10, 20, 0, 0),
       datetime.datetime(2021, 10, 20, 6, 0),
       datetime.datetime(2021, 10, 20, 12, 0),
       datetime.datetime(2021, 10, 20, 18, 0),
       datetime.datetime(2021, 10, 21, 0, 0)], dtype=object)

In [11]:
radar_df_path = [p1 for p1 in radar_dir.iterdir() if 'csv' in str(p1) and 'prd_radar' in str(p1)][0]
radar_nc_path = [p1 for p1 in radar_dir.iterdir() if 'nc' in str(p1) and 'prd_radar' in str(p1)][0]
mg_df_path = [p1 for p1 in mg_dir.iterdir() if 'csv' in str(p1) and 'prd_mogreps' in str(p1)][0]


In [46]:
mg_df = pandas.read_csv(mg_df_path)
mg_df

,realization,latitude,longitude,forecast_period,forecast_reference_time,time,cloud_area_fraction,rainfall_rate,lwe_snowfall_rate,surface_altitude,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,0,49.40625,-5.484375,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,1.991540,0.0,0.0,...,22.8125,23.2500,23.5625,24.0000,24.5000,25.0625,26.4375,28.5625,30.6875,32.3750
1,0,49.40625,-5.203125,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,2.058595,0.0,0.0,...,21.0625,21.4375,21.9375,22.6250,23.3750,24.1875,26.0625,28.0000,30.0000,31.5000
2,0,49.40625,-4.921875,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,3.144890,0.0,0.0,...,18.8125,19.2500,19.8750,20.6250,21.5000,22.5000,25.1250,27.8750,30.4375,32.2500
3,0,49.40625,-4.640625,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,3.557280,0.0,0.0,...,19.1875,19.6875,20.3125,21.0000,21.7500,22.7500,25.4375,28.8750,32.0000,33.9375
4,0,49.40625,-4.359375,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,1.756847,0.0,0.0,...,22.0625,22.6250,23.3125,24.0625,24.8125,25.7500,27.9375,31.5000,34.4375,36.1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192775,17,58.78125,1.546875,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.921875,0.000000,0.0,0.0,...,15.5000,14.9375,14.5000,14.1250,13.8750,13.7500,14.5000,14.6875,14.8125,15.8750
192776,17,58.78125,1.828125,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.937500,0.000000,0.0,0.0,...,15.3750,14.7500,14.1875,13.6875,13.1875,12.9375,13.4375,13.3750,14.0000,15.4375
192777,17,58.78125,2.109375,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.531250,0.000000,0.0,0.0,...,15.5625,14.8125,14.1250,13.5000,12.8750,12.5625,12.6875,12.4375,12.9375,14.7500
192778,17,58.78125,2.390625,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.906250,0.000000,0.0,0.0,...,15.8125,15.0625,14.2500,13.5000,12.9375,12.6875,12.6250,12.0625,12.0625,13.9375


In [45]:
radar_df = pandas.read_csv(radar_df_path)
radar_df

,time,latitude,longitude,radar_max_rain_aggregate_3hr,radar_mean_rain_aggregate_3hr,radar_max_rain_instant,radar_mean_rain_instant,fraction_sum_agg,fraction_sum_instant,radar_fraction_in_band_aggregate_3hr_0.25,radar_fraction_in_band_aggregate_3hr_2.5,radar_fraction_in_band_aggregate_3hr_7.0,radar_fraction_in_band_aggregate_3hr_10.0,radar_fraction_in_band_instant_0.25,radar_fraction_in_band_instant_2.5,radar_fraction_in_band_instant_7.0,radar_fraction_in_band_instant_10.0
0,1990-10-19 18:00:00,49.21875,-5.765625,12.796875,7.326312,14.50000,3.372883,1.002336,1.004673,0.000000,0.018692,0.810748,0.172897,0.000000,0.747664,0.235981,0.021028
1,1990-10-19 18:00:00,49.21875,-5.484375,11.567709,4.608669,8.75000,2.252409,1.004673,1.011682,0.000000,0.278037,0.719626,0.007009,0.021028,0.890187,0.100467,0.000000
2,1990-10-19 18:00:00,49.21875,-5.203125,4.867188,3.108387,5.50000,1.297863,1.000000,1.016393,0.000000,0.899297,0.100703,0.000000,0.189696,0.814988,0.011710,0.000000
3,1990-10-19 18:00:00,49.21875,-4.921875,6.333333,4.063445,5.46875,0.930887,1.000000,1.020930,0.000000,0.458140,0.541860,0.000000,0.325581,0.686047,0.009302,0.000000
4,1990-10-19 18:00:00,49.21875,-4.640625,5.497396,4.227962,8.71875,0.448817,1.000000,1.011682,0.000000,0.308411,0.691589,0.000000,0.810748,0.172897,0.028037,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10950,1990-10-21 06:00:00,58.78125,0.421875,1.622396,0.467996,1.96875,0.109049,1.000000,1.000000,0.585075,0.414925,0.000000,0.000000,0.913433,0.086567,0.000000,0.000000
10951,1990-10-21 06:00:00,58.78125,0.703125,0.502604,0.031456,0.00000,0.000000,1.000000,1.000000,0.997076,0.002924,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
10952,1990-10-21 06:00:00,58.78125,0.984375,0.536458,0.103137,0.00000,0.000000,1.000000,1.000000,0.973451,0.026549,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
10953,1990-10-21 06:00:00,58.78125,1.265625,0.914062,0.045653,0.00000,0.000000,1.000000,1.000000,0.972393,0.027607,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000


In [12]:
radar_cubelist_regridded = iris.load(str(radar_nc_path))
radar_cubelist_regridded

Average Rain In Radar Cells Within Mogreps-G Cell (mm),time,latitude,longitude
Shape,7,52,31
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x
Attributes,,,Conventions CF-1.7
Sum Of Fractions For Each Cell For Aggregate 3Hr Data (mm),time,latitude,longitude
Shape,7,52,31
Dimension coordinates,,,
time,x,-,-


In [13]:
[c1 for c1 in radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').cells()]

[Cell(point=cftime.DatetimeGregorian(1990, 10, 19, 18, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 20, 0, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 20, 6, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 20, 12, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 20, 18, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 21, 0, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 10, 21, 6, 0, 0, 0, has_year_zero=False), bound=None)]

In [14]:
[vt.timestamp() for vt in
             validity_times]

[1634662800.0,
 1634684400.0,
 1634706000.0,
 1634727600.0,
 1634749200.0,
 1634770800.0]

In [20]:
target_cube = iris.load_cube('/project/informatics_lab/precip_rediagnosis/target_cube.nc')

In [21]:
target_cube

<iris 'Cube' of rainfall_rate / (m s-1) (latitude: 52; longitude: 31)>

In [22]:
[datetime.datetime.fromtimestamp(vt1.timestamp()) for vt1 in validity_times]

[datetime.datetime(2021, 10, 19, 18, 0),
 datetime.datetime(2021, 10, 20, 0, 0),
 datetime.datetime(2021, 10, 20, 6, 0),
 datetime.datetime(2021, 10, 20, 12, 0),
 datetime.datetime(2021, 10, 20, 18, 0),
 datetime.datetime(2021, 10, 21, 0, 0)]

In [24]:
cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian')

Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian')

In [25]:
[c1 for c1 in iris.coords.DimCoord(
            [vt.timestamp() for vt in
             validity_times],
            var_name='time',
            units=target_cube.coord('time').units,
        ).cells()]

[Cell(point=cftime.DatetimeGregorian(2021, 10, 19, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 19, 23, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 5, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 11, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 23, 0, 0, 0, has_year_zero=False), bound=None)]

In [26]:
[c1 for c1 in iris.coords.DimCoord(
            [vt.timestamp() for vt in
             validity_times],
            var_name='time',
            units=radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').units,
        ).cells()]

[Cell(point=cftime.DatetimeGregorian(2021, 10, 19, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 19, 23, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 5, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 11, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(2021, 10, 20, 23, 0, 0, 0, has_year_zero=False), bound=None)]

In [47]:
dt_list1 = [dt1 for dt1 in pandas.date_range(datetime.datetime(2020,8,3,12,0), datetime.datetime(2020,8,5,12,0), freq=datetime.timedelta(hours=6)).to_pydatetime()]
dt_list1

[datetime.datetime(2020, 8, 3, 12, 0),
 datetime.datetime(2020, 8, 3, 18, 0),
 datetime.datetime(2020, 8, 4, 0, 0),
 datetime.datetime(2020, 8, 4, 6, 0),
 datetime.datetime(2020, 8, 4, 12, 0),
 datetime.datetime(2020, 8, 4, 18, 0),
 datetime.datetime(2020, 8, 5, 0, 0),
 datetime.datetime(2020, 8, 5, 6, 0),
 datetime.datetime(2020, 8, 5, 12, 0)]

In [ ]:
dt1 = [datetime.datetime(2020,8,3,12,0),
       datetime.datetime(2020,8,3,18,0),
       datetime.datetime(2020,8,4,0,0),
       datetime.datetime(2020,8,4,6,0),
       datetime.datetime(2020,8,4,12,0),
       datetime.datetime(2020,8,4,18,0),
       datetime.datetime(2020,8,4,0,0),
       datetime.datetime(2020,8,4,6,0),
       datetime.datetime(2020,8,4,12,0),
       datetime.datetime(2020,8,4,18,0),
      ]

In [41]:
[c1 for c1 in iris.coords.DimCoord(
            [cf_units.encode_time(year=dt1.year,month=dt1.month, day=dt1.day, hour=dt1.hour, minute=dt1.minute, second=0) for dt1 in dt_list1],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]


[Cell(point=cftime.DatetimeGregorian(1959, 8, 3, 12, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 3, 18, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 4, 0, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 4, 6, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 4, 12, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 4, 18, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 5, 0, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 5, 6, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1959, 8, 5, 12, 0, 0, 0, has_year_zero=False), bound=None)]

In [42]:
[c1 for c1 in iris.coords.DimCoord(
            [dt1.timestamp() for dt1 in dt_list1],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]


[Cell(point=cftime.DatetimeGregorian(1990, 8, 3, 11, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 3, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 3, 23, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 4, 5, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 4, 11, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 4, 17, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 4, 23, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 5, 5, 0, 0, 0, has_year_zero=False), bound=None),
 Cell(point=cftime.DatetimeGregorian(1990, 8, 5, 11, 0, 0, 0, has_year_zero=False), bound=None)]

In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [dt1.timestamp() for dt1 in list(pandas.date_range(datetime.datetime(2020,8,3,12,0), datetime.datetime(2020,8,5,12,0), freq=datetime.timedelta(hours=6)).to_pydatetime())],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]

In [18]:
radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').units

Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian')

In [43]:
radar_df_path

PosixPath('/scratch/shaddad/precip_rediagnosis/train_202207/202110_nswws_amber_oct/radar/prd_radar_20211019T1800Z_20211021T0600Z.csv')

In [ ]:
radar_nc_raw = iris.load_cube(str(radar_dir / 'composite_rainfall_20211020.nc'))

In [ ]:
[c1 for c1 in radar_nc_raw.coord('time').cells()][:10]

In [ ]:
def compare_time(t1, t2):
    print f'{t1.hour} {t1.hour} {t1.hour} {t1.hour} {t1.minute}'
    is_match = (t1.year == t2.year) and  (t1.month == t2.month) and  (t1.day == t2.day) and  (t1.hour== t2.hour) and  (t1.minute == t2.minute)
    return is_match


In [ ]:
list(radar_nc_raw.coord('time')[0].cells())[0].point

In [ ]:
validity_times

In [ ]:
print(validity_times[3])
radar_single_ts = radar_nc_raw.extract(iris.Constraint(time=lambda cell1: compare_time(cell1.point, validity_times[3]) ))
radar_single_ts 

In [ ]:
xarray.DataArray.from_iris(radar_single_ts).to_dataframe()['time'].value_counts()

In [ ]:
radar_df = pandas.read_csv(radar_df_path)
mg_df = pandas.read_csv(mg_df_path)

In [ ]:
radar_df

In [ ]:
iris.coordvalidity_times[2].timestamp()

In [ ]:
radar_df.time.min(), radar_df.time.max()

In [ ]:
radar_df.time

In [ ]:
radar_df.time.value_counts()

In [ ]:
mg_df

In [ ]:
mg_df.time.min(), mg_df.time.max()

In [ ]:
mg_df.time.value_counts()

In [ ]:
1565*18

In [ ]:
pandas.merge(radar_df, mg_df, on=['time','latitude','longitude'])


In [ ]:
cl1 = iris.cube.CubeList([iris.load_cube(str(f1)) for f1 in radar_root.iterdir() if 'rainfall' in f1.stem])
iris.util.equalise_attributes(cl1)
cl1.concatenate_cube()


In [ ]:
a1 = [2,5,7,4,3]
a1

In [ ]:
a1.sort()
a1